In [ ]:
#<start>only for colab users

In [ ]:
!pip install mtcnn

In [ ]:
!mkdir images

In [ ]:
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/alireza.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/ali.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/mohsen.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/muhammad.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/1.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/srttu-class.png -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/m_wrapped.png -P images

In [ ]:
!wget https://raw.githubusercontent.com/Alireza-Akhavan/deep-face-recognition/master/ArcFace.py

In [ ]:
#<end>only for colab users

In [1]:
import ArcFace
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing import image
import numpy as np
%matplotlib inline

In [2]:
model = ArcFace.loadModel()
face_detector = MTCNN()

In [3]:
def detect_face(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #mtcnn expects RGB but OpenCV read BGR
    detections = face_detector.detect_faces(img_rgb)
    detection = detections[0]
    x, y, w, h = detection["box"]
    detected_face = img[int(y):int(y+h), int(x):int(x+w)]
    return detected_face

In [4]:
def preprocess_face(img, target_size=(112,112)):
    img = cv2.imread(img)
    img = detect_face(img)
    img = cv2.resize(img, target_size)
    img_pixels = image.img_to_array(img)
    img_pixels = np.expand_dims(img_pixels, axis = 0)
    img_pixels /= 255 #normalize input in [0, 1]
    return img_pixels

In [5]:
def img_to_encoding(path):
    img = preprocess_face(path)
    return model.predict(img)[0]

In [7]:
database = {}

database["alireza"] = img_to_encoding("./images/alireza.jpg")
database["ali"] = img_to_encoding("./images/ali.jpg")
database["mohsen"] = img_to_encoding("./images/mohsen.jpg")
database["muhammad"] = img_to_encoding("./images/muhammad.jpg")

In [8]:
def EuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [23]:
verification_threshhold = 4.4

## Face Verification

In [10]:
def verify(image_path, identity, database):
    # Step 1: Compute the encoding for the image. Use img_to_encoding()
    encoding = img_to_encoding(image_path) 
    
    # Step 2: Compute distance with identity's image
    dist = EuclideanDistance(encoding, database[identity])
    
    # Step 3: Open the door if dist < verification_threshhold, else don't open
    if dist < verification_threshhold:
        print("It's " + str(identity) + ", welcome!")
    else:
        print("It's not " + str(identity) + ", please go away")
             
    return dist

<img src='images/1.jpg' style="width: 100px;"><img src='images/alireza.jpg' style="width: 100px;">

In [12]:
verify("images/1.jpg", "alireza", database)

It's alireza, welcome!


3.2458622

<img src='images/ali.jpg' style="width: 100px;"><img src='images/alireza.jpg' style="width: 100px;">

In [13]:
verify("images/ali.jpg", "alireza", database)

It's not alireza, please go away


4.855377

## Face Identification

In [14]:
def who_is_it(image_path, database):
   
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding()
    encoding = img_to_encoding(image_path)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 
    min_dist = 1000
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        dist = EuclideanDistance(encoding, db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if min_dist > dist:
            min_dist = dist
            identity = name

  
    if min_dist > verification_threshhold:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

<img src='images/srttu-class.png' style="width: 300px;">

In [16]:
who_is_it("images/srttu-class.png", database)

it's alireza, the distance is 3.835887


(3.835887, 'alireza')

<img src='images/m.jpg' style="width: 160px;">

In [18]:
who_is_it("images/m.jpg", database)

Not in the database.


(11.598342, 'alireza')

<img src='images/m_wrapped.jpg' style="width: 160px;">

In [24]:
who_is_it("images/m_wrapped.jpg", database)

it's mohsen, the distance is 4.3644657


(4.3644657, 'mohsen')